In [1]:
%load_ext autoreload
%autoreload 2
import os; print(os.getcwd())
import socket; print(socket.gethostname())

import sys
sys.path.append("../")
import glob
import re
import insrtr
import mdtraj as md 
import pandas as pd

from pathlib import Path
from glob import glob

d:\owncloud\Projekti_KI\SIDE\2022-06-10__CCRegulation\INSRTR\notebooks
ajaomen


# Load individual PDB features

In [2]:
data_files = glob('../data/output/wt/af2/[!~]*.xlsx') # Exclude excel temporary files with !~
dfs = []
for afile in data_files:
    print(afile)
    df = pd.read_excel(afile, index_col=0)
    dfs.append(df)

calc_data = pd.concat(dfs, ignore_index=True)

../data/output/wt/af2\Bgal.xlsx
../data/output/wt/af2\CarT.xlsx
../data/output/wt/af2\dCas.xlsx
../data/output/wt/af2\fLuc.xlsx
../data/output/wt/af2\Lck.xlsx
../data/output/wt/af2\mIRAK1.xlsx
../data/output/wt/af2\MyD88.xlsx
../data/output/wt/af2\ngGFP.xlsx
../data/output/wt/af2\TALEA.xlsx
../data/output/wt/af2\TEVp.xlsx


In [3]:

sheet_id = '1Bxni0ksPo4P_4vK_kraQWpU8vXE_v2pYUhbSl0pAC_U'
sheet_name = 'Sheet1'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
#experimental_data = pd.read_excel('../data/INSRTR_constructs.xlsx')
experimental_data = pd.read_csv(url)
#experimental_data
experimental_data['insert_index0']=experimental_data['insert_index_manual_fix_1']-1
experimental_data=experimental_data.set_index('Protein insert_index0'.split(), drop=False)
#Features are 0 indexed in the file
experimental_data.index.rename(['struct_name', 'resi_index0'], inplace=True)
calc_data=calc_data.set_index('struct_name resi_index0'.split(), drop=False)

In [4]:
merged_SAR = pd.merge(calc_data, experimental_data, how='left', right_index=True,  left_index=True)
#merged_SAR.to_excel('../data/INSRTR_SAR.xlsx')

# Load and merge embeddings

In [5]:
# read embeddings
data_files = glob('../data/output/embeddings/*.csv.gz')
dfs = []
for afile in data_files:
    print(afile)
    df = pd.read_csv(afile, index_col=0)
    df['struct_name'] = Path(afile).with_suffix('').stem
    df.index.rename('resi_index0', inplace=True)
    df.reset_index(drop=False, inplace=True)
    df.set_index('struct_name resi_index0'.split(), drop=True, inplace=True)
    

    #break
    dfs.append(df)

embedings = pd.concat(dfs)

embedings

../data/output/embeddings\Bgal.csv.gz
../data/output/embeddings\CarT.csv.gz
../data/output/embeddings\dCas.csv.gz
../data/output/embeddings\fLuc.csv.gz
../data/output/embeddings\Lck.csv.gz
../data/output/embeddings\mIRAK1.csv.gz
../data/output/embeddings\MyD88.csv.gz
../data/output/embeddings\ngGFP.csv.gz
../data/output/embeddings\TALEA.csv.gz
../data/output/embeddings\TEVp.csv.gz


esm2_t33_650M_0  esm2_t33_650M_1  esm2_t33_650M_2  \
struct_name resi_index0                                                      
Bgal        0                  -0.009618        -0.149050         0.086177   
            1                   0.113015        -0.127987        -0.028194   
            2                  -0.008810        -0.095047         0.102566   
            3                  -0.121956        -0.119421         0.089178   
            4                  -0.033702        -0.071638        -0.093781   
...                                  ...              ...              ...   
TEVp        238                 0.034719        -0.154053         0.059660   
            239                 0.034355        -0.154983         0.029723   
            240                 0.083200        -0.140175        -0.004161   
            241                -0.002585        -0.151745         0.090750   
            242                 0.015899        -0.065410         0.031180   

                         esm2_t33_650M_3  esm2_t33_650M_4  esm2_t33_650M_5  \
struct_name resi_index0                                                      
Bgal        0                  -0.106138        -0.011404         0.020405   
            1                  -0.232642        -0.166919         0.322973   
            2                  -0.302499        -0.020979         0.094813   
            3                   0.013324        -0.072527         0.003988   
            4                  -0.171586        -0.261692         0.138959   
...                                  ...              ...              ...   
TEVp        238                 0.120289         0.033725         0.030815   
            239                 0.173692         0.079296         0.125224   
            240                 0.129351        -0.006860         0.081574   
            241                 0.012099         0.080732         0.045014   
            242                 0.079401        -0.056141         0.023695   

                         esm2_t33_650M_6  esm2_t33_650M_7  esm2_t33_650M_8  \
struct_name resi_index0                                                      
Bgal        0                   0.045280        -0.174383        -0.009581   
            1                  -0.011954         0.038781         0.068682   
            2                  -0.103842         0.040719        -0.043413   
            3                  -0.002809         0.082841        -0.064119   
            4                  -0.176938         0.076468        -0.157592   
...                                  ...              ...              ...   
TEVp        238                 0.114116        -0.096812         0.212066   
            239                 0.102238        -0.036995         0.285801   
            240                 0.036021        -0.052163         0.208156   
            241                -0.083584         0.079751         0.050780   
            242                -0.079548         0.026262         0.115209   

                         esm2_t33_650M_9  ...  esm2_t33_650M_1270  \
struct_name resi_index0                   ...                       
Bgal        0                  -0.059337  ...           -0.013213   
            1                  -0.258103  ...            0.078438   
            2                  -0.076283  ...            0.149049   
            3                   0.147056  ...           -0.051038   
            4                   0.006477  ...            0.088386   
...                                  ...  ...                 ...   
TEVp        238                -0.088674  ...           -0.022891   
            239                -0.182656  ...            0.060547   
            240                -0.218216  ...            0.012291   
            241                 0.085848  ...            0.171691   
            242                -0.107031  ...            0.046532   

                         esm2_t33_650M_1271  esm2_t33_650M_1272  \
struct_name res

In [6]:
merged_SAR = pd.merge(merged_SAR, embedings, how='left', right_index=True,  left_index=True)
merged_SAR

struct_name  resi_loop_index0  loop_index0  \
struct_name resi_index0                                              
Bgal        2                  Bgal                 0           91   
            3                  Bgal                 1           91   
            4                  Bgal                 2           91   
            6                  Bgal                 0           92   
            7                  Bgal                 1           92   
...                             ...               ...          ...   
ngGFP       238               ngGFP                 3           17   
            239               ngGFP                 4           17   
            240               ngGFP                 0           22   
            241               ngGFP                 1           22   
            242               ngGFP                 2           22   

                         resi_index0  resi_distance_to_N_term_A  \
struct_name resi_index0                                           
Bgal        2                      2                   0.000000   
            3                      3                   3.874204   
            4                      4                   5.398853   
            6                      6                   0.000000   
            7                      7                   3.878922   
...                              ...                        ...   
ngGFP       238                  238                   6.739091   
            239                  239                   9.995601   
            240                  240                   0.000000   
            241                  241                   3.885818   
            242                  242                   5.465230   

                         resi_distance_to_C_term_A resi_type resi_dssp  \
struct_name resi_index0                                                  
Bgal        2                             5.398853         L         H   
            3                             3.863977         Q         H   
            4                             0.000000         R         H   
            6                             5.616211         D         H   
            7                             3.934944         W         H   
...                                            ...       ...       ...   
ngGFP       238                           3.911345         M         L   
            239                           0.000000         G         L   
            240                           5.465230         M         H   
            241                           3.781210         D         H   
            242                           0.000000         E         H   

                        prev_resi_type prev_resi_dssp  ... esm2_t33_650M_1270  \
struct_name resi_index0                                ...                      
Bgal        2                        V              L  ...           0.149049   
            3                        L              H  ...          -0.051038   
            4                        Q              H  ...           0.088386   
            6                        R              L  ...          -0.057354   
            7                        D              H  ...          -0.008862   
...                                ...            ...  ...                ...   
ngGFP       238                      V              L  ...           0.019142   
            239                      M              L  ...           0.054527   
            240                      G              L  ...           0.021638   
            241                      M              H  ...          -0.012268   
            242                      D              H  ...           0.176489   

                        esm2_t33_650M_1271  esm2_t33_650M_1272  \
struct_name resi_index0                                          
Bgal        2                    -0.264989           -0.078383   
            3             

# Load NMA analysis

In [7]:
# read embeddings
data_files = glob('../data/output/wt/af2/NMA/*/displacement/fluctuations.txt')
dfs = []
for afile in data_files:
    print(afile)
    df = pd.read_csv(afile, sep="\t" )
    #read folder name (two folders up)
    struct_name = Path(afile).parent.parent.stem
    #print(struct_name)
    df['struct_name'] = struct_name
    df['resi_index0'] = df['index']-1
    df.drop(columns=['index'], inplace=True)
    df.set_index('struct_name resi_index0'.split(), drop=True, inplace=True)
    
    dfs.append(df)

nma = pd.concat(dfs)
nma


../data/output/wt/af2/NMA\Bgal\displacement\fluctuations.txt
../data/output/wt/af2/NMA\CarT\displacement\fluctuations.txt
../data/output/wt/af2/NMA\dCas\displacement\fluctuations.txt
../data/output/wt/af2/NMA\fLuc\displacement\fluctuations.txt
../data/output/wt/af2/NMA\Lck\displacement\fluctuations.txt
../data/output/wt/af2/NMA\mIRAK1\displacement\fluctuations.txt
../data/output/wt/af2/NMA\MyD88\displacement\fluctuations.txt
../data/output/wt/af2/NMA\ngGFP\displacement\fluctuations.txt
../data/output/wt/af2/NMA\TALEA\displacement\fluctuations.txt
../data/output/wt/af2/NMA\TEVp\displacement\fluctuations.txt


fluctuation
struct_name resi_index0             
Bgal        0                  0.449
            1                  0.116
            2                  0.065
            3                  0.102
            4                  0.076
...                              ...
TEVp        238                0.654
            239                1.176
            240                4.223
            241               17.695
            242               75.519

[6335 rows x 1 columns]

In [8]:

merged_SAR = pd.merge(merged_SAR, nma, how='left', right_index=True,  left_index=True)
merged_SAR

struct_name  resi_loop_index0  loop_index0  \
struct_name resi_index0                                              
Bgal        2                  Bgal                 0           91   
            3                  Bgal                 1           91   
            4                  Bgal                 2           91   
            6                  Bgal                 0           92   
            7                  Bgal                 1           92   
...                             ...               ...          ...   
ngGFP       238               ngGFP                 3           17   
            239               ngGFP                 4           17   
            240               ngGFP                 0           22   
            241               ngGFP                 1           22   
            242               ngGFP                 2           22   

                         resi_index0  resi_distance_to_N_term_A  \
struct_name resi_index0                                           
Bgal        2                      2                   0.000000   
            3                      3                   3.874204   
            4                      4                   5.398853   
            6                      6                   0.000000   
            7                      7                   3.878922   
...                              ...                        ...   
ngGFP       238                  238                   6.739091   
            239                  239                   9.995601   
            240                  240                   0.000000   
            241                  241                   3.885818   
            242                  242                   5.465230   

                         resi_distance_to_C_term_A resi_type resi_dssp  \
struct_name resi_index0                                                  
Bgal        2                             5.398853         L         H   
            3                             3.863977         Q         H   
            4                             0.000000         R         H   
            6                             5.616211         D         H   
            7                             3.934944         W         H   
...                                            ...       ...       ...   
ngGFP       238                           3.911345         M         L   
            239                           0.000000         G         L   
            240                           5.465230         M         H   
            241                           3.781210         D         H   
            242                           0.000000         E         H   

                        prev_resi_type prev_resi_dssp  ... esm2_t33_650M_1271  \
struct_name resi_index0                                ...                      
Bgal        2                        V              L  ...          -0.264989   
            3                        L              H  ...          -0.104491   
            4                        Q              H  ...          -0.149562   
            6                        R              L  ...           0.028528   
            7                        D              H  ...          -0.235685   
...                                ...            ...  ...                ...   
ngGFP       238                      V              L  ...          -0.050436   
            239                      M              L  ...           0.101670   
            240                      G              L  ...          -0.063160   
            241                      M              H  ...           0.050420   
            242                      D              H  ...          -0.042886   

                        esm2_t33_650M_1272  esm2_t33_650M_1273  \
struct_name resi_index0                                          
Bgal        2                    -0.078383            0.107079   
            3             

In [9]:
#merged_SAR.query('esm2_t33_650M_1271.isnull()')
#merged_SAR.query('fluctuation.isnull()')

# Load MSA conservation


In [10]:
data_files = glob('../data/output/conservation/*.txt')
dfs = []
for afile in data_files:
    print(afile)
    df = pd.read_csv(afile, sep="," )
    #read folder name (two folders up)
    df.columns='resi_index1 relative_MSA_conservation'.split()
    df['resi_index0']=df['resi_index1']-1
    df.drop(columns=['resi_index1'], inplace=True)
    struct_name = Path(afile).stem
    df['struct_name'] = struct_name
    df.set_index('struct_name resi_index0'.split(), drop=True, inplace=True)
    #print(struct_name)
    #display(df.head())
    
    dfs.append(df)

conservation = pd.concat(dfs)
conservation

../data/output/conservation\Bgal.txt
../data/output/conservation\CarT.txt
../data/output/conservation\dCas.txt
../data/output/conservation\fLuc.txt
../data/output/conservation\Lck.txt
../data/output/conservation\mIRAK1.txt
../data/output/conservation\MyD88.txt
../data/output/conservation\ngGFP.txt
../data/output/conservation\TALEA.txt
../data/output/conservation\TEVp.txt


relative_MSA_conservation
struct_name resi_index0                           
Bgal        0                             2.981978
            1                             0.908764
            2                             0.834617
            3                             0.756893
            4                             0.697655
...                                            ...
TEVp        238                           0.587478
            239                           0.524740
            240                           0.942390
            241                           0.432547
            242                           0.707727

[6871 rows x 1 columns]

In [11]:

merged_SAR = pd.merge(merged_SAR, conservation, how='left', right_index=True,  left_index=True)
merged_SAR

struct_name  resi_loop_index0  loop_index0  \
struct_name resi_index0                                              
Bgal        2                  Bgal                 0           91   
            3                  Bgal                 1           91   
            4                  Bgal                 2           91   
            6                  Bgal                 0           92   
            7                  Bgal                 1           92   
...                             ...               ...          ...   
ngGFP       238               ngGFP                 3           17   
            239               ngGFP                 4           17   
            240               ngGFP                 0           22   
            241               ngGFP                 1           22   
            242               ngGFP                 2           22   

                         resi_index0  resi_distance_to_N_term_A  \
struct_name resi_index0                                           
Bgal        2                      2                   0.000000   
            3                      3                   3.874204   
            4                      4                   5.398853   
            6                      6                   0.000000   
            7                      7                   3.878922   
...                              ...                        ...   
ngGFP       238                  238                   6.739091   
            239                  239                   9.995601   
            240                  240                   0.000000   
            241                  241                   3.885818   
            242                  242                   5.465230   

                         resi_distance_to_C_term_A resi_type resi_dssp  \
struct_name resi_index0                                                  
Bgal        2                             5.398853         L         H   
            3                             3.863977         Q         H   
            4                             0.000000         R         H   
            6                             5.616211         D         H   
            7                             3.934944         W         H   
...                                            ...       ...       ...   
ngGFP       238                           3.911345         M         L   
            239                           0.000000         G         L   
            240                           5.465230         M         H   
            241                           3.781210         D         H   
            242                           0.000000         E         H   

                        prev_resi_type prev_resi_dssp  ... esm2_t33_650M_1272  \
struct_name resi_index0                                ...                      
Bgal        2                        V              L  ...          -0.078383   
            3                        L              H  ...           0.038592   
            4                        Q              H  ...          -0.342901   
            6                        R              L  ...          -0.058901   
            7                        D              H  ...           0.124295   
...                                ...            ...  ...                ...   
ngGFP       238                      V              L  ...          -0.128621   
            239                      M              L  ...           0.085356   
            240                      G              L  ...          -0.124949   
            241                      M              H  ...           0.049740   
            242                      D              H  ...           0.199773   

                        esm2_t33_650M_1273  esm2_t33_650M_1274  \
struct_name resi_index0                                          
Bgal        2                     0.107079            0.098752   
            3             

In [ ]:
#merged_SAR.query('relative_MSA_conservation.isnull()')

In [13]:
merged_SAR.to_excel('../data/INSRTR_SAR.xlsx')

In [ ]:
merged_SAR.to_csv('../data/INSRTR_SAR.csv.gz')